In [0]:
%run "../setup/functions"

In [0]:
dbutils.widgets.text("start_date", "2020-04-22")
start_date = dbutils.widgets.get("start_date")

In [0]:
dbutils.widgets.text("terminate_date", "2020-04-27")
end_date = dbutils.widgets.get("terminate_date")

##Fetching Data from Bronze and Transforming

In [0]:
state = "new york"
bronze_df = spark.read.format("json").load(f"{bronze_folder_path}{state.replace(' ', '')}")

In [0]:
filtered_df = bronze_df.filter(bronze_df.county=='New York')

In [0]:
df_selected = filtered_df.select(col('county'),
                             col('confirmed_daily'),
                             col('date'),
                             col('deaths_daily'))      

In [0]:
 results_final_df = df_selected\
  .withColumnRenamed("county", "MUNICIPIO")\
  .withColumnRenamed("confirmed_daily", "CONFIRMADOS_NY")\
  .withColumnRenamed("deaths_daily", "MORTES_NY")\
  .withColumn('CHAVE_DATA', date_format(col('date'),"yyyyMM"))\
  .withColumn('DIA', date_format(col('date'),"dd"))\
  .drop('date')

##Writing into Silver Layer

In [0]:
from delta.tables import DeltaTable
if (DeltaTable.isDeltaTable(spark, f"{silver_folder_path}{state.replace(' ', '')}")):
    deltaTable = DeltaTable.forPath(spark, f"{silver_folder_path}{state.replace(' ', '')}")
    deltaTable.alias("tgt").merge(
    results_final_df.alias("src"),
     "tgt.CHAVE_DATA = src.CHAVE_DATA AND tgt.DIA = src.DIA") \
    .whenMatchedUpdateAll()\
    .whenNotMatchedInsertAll()\
    .execute()
else:
    results_final_df.write.mode("overwrite")\
    .partitionBy("CHAVE_DATA")\
    .format("delta")\
    .save(f"{silver_folder_path}{state.replace(' ', '')}")


In [0]:
state = "new york"
display(spark.read.format("delta").load(f"{silver_folder_path}{state.replace(' ', '')}"))